In [ ]:
import pandas as pd
import numpy as np
import regex as re
from pathlib import Path
import glob


## Load Data

In [3]:
def file_paths_and_names(year):
    p = Path('data')
    files = list(p.glob(f"*_{year}.csv"))
    file_names = [re.findall(r'[a-zA-Z]+', i.stem)[0] for i in files]
    # print(file_names)
    file_paths = [i for i in files]
    return file_names,file_paths
years = ['19', '20', '21', '22', '23']
files_by_year = {year: dict(zip(*file_paths_and_names(year))) for year in years}


In [82]:
def load_eventlink_file(file,year):
    return pd.read_csv(file,usecols=['DUPERSID','CONDIDX','EVNTIDX','EVENTYPE'])

In [5]:
def load_prescription_file(file,year):
    tot_exp = 'RXXP'+year+'X'
    oop_doc = 'RXSF'+year+'X'
    mdcare_doc = 'RXMR'+year+'X'
    mdcaid_doc = 'RXMD'+year+'X'
    priv_doc = 'RXPV'+year+'X'
    WT = 'PERWT'+year+'F'
    return pd.read_csv(file,usecols=['DUPERSID','RXRECIDX','LINKIDX','TC1S1_1','DIABEQUIP',tot_exp,oop_doc,mdcare_doc,mdcaid_doc,priv_doc,WT])


In [6]:
def load_conditions_file(file,year):
    WT = 'PERWT'+year+'F'
    return pd.read_csv(file,usecols = ['DUPERSID','CONDIDX','ICD10CDX',WT])

In [7]:
def load_inpatient_file(file,year):
    tot_exp = 'IPXP'+year+'X'
    oop_doc = 'IPDSF'+year+'X'
    oop_fac = 'IPFSF'+year+'X'
    mdcare_doc = 'IPDMR'+year+'X'
    mdcaid_doc = 'IPDMD'+year+'X'
    priv_doc = 'IPDPV'+year+'X'
    mdcare_fac = 'IPFMR'+year+'X'
    mdcaid_fac = 'IPFMD'+year+'X'
    priv_fac = 'IPFPV'+year+'X'
    WT = 'PERWT'+year+'F'
    return pd.read_csv(file,usecols=['DUPERSID','EVNTIDX',tot_exp,oop_doc,oop_fac,mdcare_doc,mdcare_fac,mdcaid_doc,mdcaid_fac,priv_doc,priv_fac,WT])

In [8]:
def load_outpatient_file(file,year):
    tot_exp = 'OPXP'+year+'X'
    oop_doc = 'OPDSF'+year+'X'
    oop_fac = 'OPFSF'+year+'X'
    mdcare_doc = 'OPDMR'+year+'X'
    mdcaid_doc = 'OPDMD'+year+'X'
    priv_doc = 'OPDPV'+year+'X'
    mdcare_fac = 'OPFMR'+year+'X'
    mdcaid_fac = 'OPFMD'+year+'X'
    priv_fac = 'OPFPV'+year+'X'
    WT = 'PERWT'+year+'F'
    return pd.read_csv(file,usecols=['DUPERSID','EVNTIDX',tot_exp,oop_doc,oop_fac,mdcare_doc,mdcare_fac,mdcaid_doc,mdcaid_fac,priv_doc,priv_fac,WT])


In [9]:
def load_er_file(file,year):
    tot_exp = 'ERXP'+year+'X'
    oop_doc = 'ERDSF'+year+'X'
    oop_fac = 'ERFSF'+year+'X'
    mdcare_doc = 'ERDMR'+year+'X'
    mdcaid_doc = 'ERDMD'+year+'X'
    priv_doc = 'ERDPV'+year+'X'
    mdcare_fac = 'ERFMR'+year+'X'
    mdcaid_fac = 'ERFMD'+year+'X'
    priv_fac = 'ERFPV'+year+'X'
    WT = 'PERWT'+year+'F'
    return pd.read_csv(file,usecols=['DUPERSID','EVNTIDX',tot_exp,oop_doc,oop_fac,mdcare_doc,mdcare_fac,mdcaid_doc,mdcaid_fac,priv_doc,priv_fac,WT])

In [10]:
def load_office_file(file,year):
    tot_exp = 'OBXP'+year+'X'
    oop_doc = 'OBSF'+year+'X'
    mdcare_doc = 'OBMR'+year+'X'
    mdcaid_doc = 'OBMD'+year+'X'
    priv_doc = 'OBPV'+year+'X'
    WT = 'PERWT'+year+'F'
    return pd.read_csv(file,usecols=['DUPERSID','EVNTIDX',tot_exp,oop_doc,mdcare_doc,mdcaid_doc,priv_doc,WT])

In [11]:
def load_home_file(file,year):
    tot_exp = 'HHXP'+year+'X'
    oop_doc = 'HHSF'+year+'X'
    mdcare_doc = 'HHMR'+year+'X'
    mdcaid_doc = 'HHMD'+year+'X'
    priv_doc = 'HHPV'+year+'X'
    WT = 'PERWT'+year+'F'
    return pd.read_csv(file,usecols=['DUPERSID','EVNTIDX',tot_exp,oop_doc,mdcare_doc,mdcaid_doc,priv_doc,WT])   

In [12]:
def load_fyc(file,year):
    tot_exp = 'TOTEXP'+year
    return pd.read_csv(file,usecols=['DUPERSID',tot_exp])

In [13]:
def rename_cols(df):
    old_cols = df.columns
    new_cols = [re.sub('[0-9]','',i) for i in old_cols]
    return df.rename(columns=dict(zip(old_cols,new_cols)))

In [15]:
def load_all_files(year):
    cond =  rename_cols(load_conditions_file(files_by_year[year]['conditions'],year))
    out = rename_cols(load_outpatient_file(files_by_year[year]['outpatient'],year))
    prescription =  rename_cols(load_prescription_file(files_by_year[year]['prescriptions'],year)) 
    inpatient =  rename_cols(load_inpatient_file(files_by_year[year]['inpatient'],year))    
    home =  rename_cols(load_home_file(files_by_year[year]['home'],year))   
    office =  rename_cols(load_office_file(files_by_year[year]['office'],year))
    er =  rename_cols(load_er_file(files_by_year[year]['er'],year)) 
    eventlink = load_eventlink_file(files_by_year[year]['eventlink'],year)
    fyc = rename_cols(load_fyc(files_by_year[year]['fyc'],year))
    return [cond,out,prescription,inpatient,home,office,er,eventlink,fyc]
      

In [ ]:
#load all files into dataframes and store in dictionary by year and event type 
dfs_by_year = {}
df_names = ['cond','out','prescription','inpatient','home','office','er','eventlink','fyc']
for year in years:
    dfs_by_year[year] = dict(zip(df_names,load_all_files(year)))


## conditions

In [ ]:
#Condition index and person weight for diabetes 
def condidx_diabetes(df):
    diab_df = df[df['ICDCDX']=='E11'][['CONDIDX','PERWTF']]
    diab_df['CONDIDX'] = diab_df['CONDIDX'].astype(str)
    return diab_df


In [84]:
def get_diabetes_conditions(year):
    return condidx_diabetes(dfs_by_year[year]['cond'])
cond_diab = {}
for year in years:
    cond_diab[year]=get_diabetes_conditions(year)

## Process all events

In [85]:
def create_event_diabetes(event,year):
    df = dfs_by_year[year][event]
    evntlnk = dfs_by_year[year]['eventlink']
    evntlnk['EVNTIDX'] = evntlnk['EVNTIDX'].astype(str)
    evntlnk['CONDIDX'] = evntlnk['CONDIDX'].astype(str)
    # return cond_diab[year]
    #use eventlink file to link conditions and prescriptions 
    evn_diabetes = pd.merge(left=evntlnk,right=cond_diab[year],on=['CONDIDX'])
    df['EVNTIDX'] = df['EVNTIDX'].astype(str)
    diabetes = pd.merge(left=evn_diabetes,right=df,on=['EVNTIDX','DUPERSID','PERWTF'])
    return diabetes


In [ ]:
#links prescription dataset to diabetes conditions 
def create_prescription_diabetes(year):
    evntlnk = dfs_by_year[year]['eventlink']
    p_df = dfs_by_year[year]['prescription']
    cond_df = cond_diab[year]
    evntlnk['LINKIDX'] = evntlnk['EVNTIDX'].astype(str)
    evntlnk['CONDIDX'] = evntlnk['CONDIDX'].astype(str)
    
    #use eventlink file to link conditions and prescriptions 
    evn_diabetes = pd.merge(left=evntlnk,right=cond_df,on=['CONDIDX'])
    p_df['LINKIDX'] = p_df['LINKIDX'].astype(str)
    p_diabetes = pd.merge(left=evn_diabetes,right=p_df,on=['LINKIDX','DUPERSID','PERWTF'])
    return p_diabetes


In [ ]:
def event_diabetes_person_sum(event,year):
    
    if event == 'prescription':
        df = create_prescription_diabetes(year)
    else:
        df = create_event_diabetes(event,year)

    col_sub = ['XPX','SFX','MRX','MDX','PVX']
    all_cols = dfs_by_year[year][event].columns
    payment_columns = all_cols[all_cols.str.contains('|'.join(col_sub))]
    df[payment_columns] = df[payment_columns].apply(pd.to_numeric, errors='coerce')
    # sum_payment_cols = dict(zip(payment_columns,['sum' for i in range(len(payment_columns))]))
    sum_payments = df.groupby('DUPERSID',as_index=False)[payment_columns].sum()
    # .agg(sum_payment_cols)

    sum_payments= pd.merge(left = sum_payments, right= df[['DUPERSID','PERWTF']].drop_duplicates(), on='DUPERSID')
    sum_payments[payment_columns] = sum_payments[payment_columns].multiply(sum_payments['PERWTF'],axis='index')
    return sum_payments,payment_columns


## Save Files

In [ ]:
#save all prescriptions associated with diabetes for all years,unweighted to analyze by prescription type later 
prescription_dfs = []
for year in years:
    p_df = create_prescription_diabetes(year)
    p_df['year'] = '20'+year
    p_df.drop(columns = ['EVNTIDX'],inplace=True)
    prescription_dfs.append(p_df)
unweighted_prescription_years = pd.concat(prescription_dfs,ignore_index=True)
unweighted_prescription_years.to_csv('data/processed_data/unweighted_diabetes_prescriptions.csv',index=False)

In [ ]:
#save weighted events and melt tables to tidy format 
events = ['out','inpatient','home','office','er','prescription']
long_df = []
for e in events:
    event_dfs = []
    for year in years:
        event_df,payment_columns = event_diabetes_person_sum(e,year)
        event_df['year'] = '20'+year
        event_dfs.append(event_df)
        l_df = pd.melt(event_df,id_vars=['DUPERSID','year','PERWTF'],value_vars=payment_columns)
        long_df.append(l_df)
    file_name = f"weighted_{e}_sums"
    weighted_sums = pd.concat(event_dfs,ignore_index=True)
    weighted_sums.to_csv(f'data/processed_data/{file_name}.csv',index=False)
all_events = pd.concat(long_df,ignore_index=True)
all_events.to_csv('data/processed_data/all_events.csv',index=False)
    # print(file_name)
    

In [ ]:
#save full year files 
fyc_dfs = []
for year in years:
    f_df = dfs_by_year[year]['fyc']
    f_df['year'] = '20'+year
    fyc_dfs.append(f_df)
all_fycs = pd.concat(fyc_dfs,ignore_index=True)
all_fycs.to_csv('data/processed_data/all_fycs.csv',index=False)

## Analysis


### Prescription spending totals

In [ ]:
#testing prescription payment totals 
p_d = create_prescription_diabetes('23')
print(len(p_d))
p_d = p_d[p_d['DIABEQUIP']==1]
col_sub = ['XPX','SFX','MRX','MDX','PVX']
all_cols = p_d.columns
payment_columns = all_cols[all_cols.str.contains('|'.join(col_sub))]
p_d[payment_columns] = p_d[payment_columns].apply(pd.to_numeric, errors='coerce')

sum_payments = p_d.groupby('DUPERSID',as_index=False)[payment_columns].sum()
sum_payments= pd.merge(left = sum_payments, right= p_d[['DUPERSID','PERWTF','TCS_']].drop_duplicates(), on='DUPERSID')
sum_payments[payment_columns] = sum_payments[payment_columns].multiply(sum_payments['PERWTF'],axis='index')
sum_payments

sum_payments['RXXPX'].sum()

24242


np.float64(6520011226.207245)

### avg per person spending over all events

In [110]:
# e_d,payment_columns = event_diabetes_person_sum('office','22')
events = ['out','inpatient','home','office','er','prescription']
df_22 = []
wts_22 = []
for event in events:
    e_d,payment_columns = event_diabetes_person_sum(event,'22')
    col_sub = ['XPX']
    all_cols = e_d.columns
    payment_columns = all_cols[all_cols.str.contains('|'.join(col_sub))]
    sum_e = e_d[payment_columns].sum().sum()
    wts_22.append(e_d[['DUPERSID','PERWTF']])
    df_22.append(sum_e)
# e_d.pivot(payment_columns)
# pay = [i for i in payment_columns]
# wts_22
all_wts = pd.concat(wts_22,ignore_index=True)
all_wts.drop_duplicates(subset=['DUPERSID'],inplace=True)
sum_wts = all_wts['PERWTF'].sum()
sum(df_22)/sum_wts
sum(df_22)


np.float64(158904611041.54312)

### spend by insurance

In [104]:
events_22 = all_events[all_events['year']=='2022']
events_22[events_22['variable'].str.contains('SFX')]['value'].sum()

np.float64(10606505388.319036)